## Using OpenKIM models in LAMMPS

This notebook demonstrates use of OpenKIM models in LAMMPS and some utilities available as LAMMPS subcommands. Documentation on the implementation of these features in LAMMPS can be found here: [LAMMPS `kim` command](https://docs.lammps.org/kim_commands.html). Subsections of this documentation are referenced thoughout the notebook where relevant.

For comparison, we begin with a non-KIM LAMMPS script for a single energy/force evaluation (i.e. a `run 0` script) of bulk silicon using the 1985 Stilinger-Weber potential. A user running this simulation would need to obtain their own parameter file (this one is shipped with LAMMPS) and specify the lattice constant in the input script:

<button data-commandLinker-command="docmanager:open" data-commandlinker-args='{"path":"openkim-demo/lammps/in.Si.SW.lammps"}'>View `in.Si.SW.lammps` in editor</button>

Run the computation:

In [ ]:
!lmp -in in.Si.sw.lammps

In [ ]:
from ase.io import read
from ase.visualize import view
atoms = read('Si.sw.data',format="lammps-data",style="atomic",Z_of_type={1:14}).repeat(2)
atoms.set_cell(0.5*atoms.get_cell())
view(atoms,viewer='ngl')

Now let's look at the same simulation set up with OpenKIM. Initialization and setup of the atomic interactions is handled by the `kim init` and `kim interactions` commands ([LAMMPS documentation](https://docs.lammps.org/kim_commands.html#using-openkim-ims-with-lammps-kim-init-kim-interactions))

<button data-commandLinker-command="docmanager:open" data-commandlinker-args='{"path":"openkim-demo/lammps/in.Si.sw.kim.lammps"}'>View `in.Si.SW.kim.lammps` in editor</button>

Run the computation:

In [ ]:
!lmp -in in.Si.SW.kim.lammps

Differences in the KIM and LAMMPS implementation of the Stilinger-Weber potential cause small differences numerical differences, but the results of the two computations are nearly identical. Note also that KIM prints brief citation information for the potential to the screen and detailed BibTeX-formatted citations to the LAMMPS log file.

You may have noticed that there is a small, but finite pressure in this configuration. This is because the 5.43 angstrom value we used for the lattice constant is not exactly what the Stilinger-Weber potential predicts. We can use the `kim query` LAMMPS command to obtain the correct equilibrium lattice constant for this potential from the OpenKIM repository:

<button data-commandLinker-command="docmanager:open" data-commandlinker-args='{"path":"openkim-demo/lammps/in.Si.sw.kim.query.lammps"}'>View `in.Si.SW.kim.query.lammps` in editor</button>

If we run the simulation, we see that the pressure is now nearly zero and the energy is slightly lower:

In [ ]:
!lmp -in in.Si.SW.kim.query.lammps

Let's try to run the same bulk silicon calculation with a different OpenKIM model, in this case a ReaxFF potential:

<button data-commandLinker-command="docmanager:open" data-commandlinker-args='{"path":"openkim-demo/lammps/in.Si.reax.kim.query.lammps"}'>View `in.Si.SW.kim.query.lammps` in editor</button>

